In [1]:
pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import chess
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

csv_file = "/content/drive/My Drive/evaluations_depth_4.csv"
 #Disable XLA JIT compilation
tf.config.optimizer.set_jit(False)

# Set cuDNN autotune environment variable to avoid problematic algorithms
import os
os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'
def create_model():
    input_layer = Input(shape=(14, 8, 8))

    conv_layer_1 = Conv2D(128, (3, 3), padding='same', activation='relu')(input_layer)
    conv_layer_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_layer_1)
    conv_layer_3 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_layer_2)


    flatten_layer = Flatten()(conv_layer_2)

    dense_layer_1 = Dense(128, activation='relu')(flatten_layer)
    output_layer = Dense(1, activation='linear')(dense_layer_1)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(5e-4), loss="mean_squared_error")
    model.summary()
    return model

board_positions = {chr(i): i - ord('a') for i in range(ord('a'), ord('h') + 1)}

def standardize_evaluation(value, min_value, max_value):
    try:
        value = float(value)
    except ValueError:
        cleaned_value = value.lstrip('#')
        try:
            numeric_value = float(cleaned_value)
            value = numeric_value * 15000
        except ValueError:
            print("cannot clean")
    standardized_value = (value - min_value) / (max_value - min_value)
    return standardized_value

def square_to_index(square):
    letter = chess.square_name(square)
    row = 8 - int(letter[1])
    column = board_positions[letter[0]]
    return row, column

def boards_to_matrices_batch(boards):
    batch_size = len(boards)
    board_3d_batch = np.zeros((batch_size, 14, 8, 8), dtype=np.int8)

    for b_idx, board in enumerate(boards):
        for piece in chess.PIECE_TYPES:
            for square in board.pieces(piece, chess.WHITE):
                index = np.unravel_index(square, (8, 8))
                board_3d_batch[b_idx, piece - 1, 7 - index[0], index[1]] = 1
            for square in board.pieces(piece, chess.BLACK):
                index = np.unravel_index(square, (8, 8))
                board_3d_batch[b_idx, piece + 5, 7 - index[0], index[1]] = 1

        aux = board.turn
        board.turn = chess.WHITE
        for move in board.legal_moves:
            i, j = np.unravel_index(move.to_square, (8, 8))
            board_3d_batch[b_idx, 12, 7 - i, j] = 1

        board.turn = chess.BLACK
        for move in board.legal_moves:
            i, j = np.unravel_index(move.to_square, (8, 8))
            board_3d_batch[b_idx, 13, 7 - i, j] = 1

        board.turn = aux

    return board_3d_batch

def process_batch(batch):
    valid_rows = batch.dropna(subset=['fen', 'evaluation'])
    y_evaluation = valid_rows['evaluation'].apply(standardize_evaluation, args=(-10000, 10000)).values

    feature_boards = []
    for fen in valid_rows['fen']:
        board = chess.Board(fen)
        feature_boards.append(board)
    feature_board = boards_to_matrices_batch(feature_boards)
    return feature_board, y_evaluation

def data_generator_from_csv(csv_file, batch_size):
    chunk_size = batch_size
    while True:
        for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
            processed_data = process_batch(chunk)
            if processed_data:
                x_data, y_data = processed_data
                yield x_data, y_data

def tf_data_generator(csv_file, batch_size):
    generator = lambda: data_generator_from_csv(csv_file, batch_size)
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 14, 8, 8), dtype=tf.float32),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )
    )
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

batch_size = 512  # Adjust batch size if necessary
train_dataset = tf_data_generator(csv_file, batch_size)

model = create_model()
epochs = 100  # One epoch for each batch

num_samples = sum(1 for row in open(csv_file)) - 1  # Subtract 1 for header
steps_per_epoch = (num_samples + batch_size - 1) // batch_size

checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/My Drive/models_save/model_checkpoint_depth4_{epoch:02d}.h5',  # Filename pattern to save the model
    save_freq='epoch',  # Save the model at the end of every epoch
    save_best_only=False  # You can set this to True to save only the best model
)

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=[
        ReduceLROnPlateau(monitor="loss", patience=10),
        EarlyStopping(monitor="loss", patience=15, min_delta=1e-4),
        checkpoint_callback
    ]
)

plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d (Conv2D)             (None, 14, 8, 128)        9344      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 8, 128)        147584    
                                                                 
 flatten (Flatten)           (None, 14336)             0         
                                                                 
 dense (Dense)               (None, 128)               1835136   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1992193 (7.60 MB)
Trainable params: 1992193 (7.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9524/9524 [==============================] - 17033s 2s/step - loss: 3.5657e-05 - lr: 5.0000e-04


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/100
5046/9524 [==============>...............] - ETA: 2:13:57 - loss: 1.7848e-05